<h1 style="color:brown;">  Recurrent neural nets</h1> 

![Looping network](./img/RNN_colah.png)

##### RNNs can produce amazing results <a href ="http://karpathy.github.io/2015/05/21/rnn-effectiveness/">blog</a>

### Lesson plan 
1. Why classic neural nets are not enough?
2. Word embeddings - word2vec
3. Categorical embeddings
4. RNN 
5. Takeaways
6. Hands on word2vec

In [3]:
import numpy as np

### Classic nets vs. RNN's

Classic:
    - Inputs and outputs must be fixed-sized vectors
    - No idea of location or time 

RNNs: 

![](./img/diags.jpeg)

### Word embeddings

#### N-gram

-Which word produces the highest probability to be next given we have seen n specific other words before

In [ ]:
Words: Thank, you, Hello, goodbye

In [ ]:
If we have 4 words and we are looking at 2-gram? 
    Example: no. of times Thank you occurs divided by number of times Thank occurs

We need to calculate the probabilty of 
 - Thank Hello
 - Thank you
 - Thank goodbye
 - Thank Thank

So we needed to do 4 calculations

In [17]:
def how_many_calc_to_do(gram, voc_size):
    '''This function needs to calculate all combos 
    of words'''
    
    return np.prod(np.repeat(voc_size, gram))

In [15]:
how_many_calc_to_do(7, 10000)
# Notice that this is only an approxiamtion and it can be implemented in more efficient ways.

4477988020393345024

![](./img/one_hot_encoding_distance_on_3d.png)

#### Insight I: 
    we can actually just turn each word to a random vector sized 100/200/300, 
    train a classic neural net to predict the next word and update both the weights and the random vectors.
    You can think of it as just another layer of weights multiplying the one hot encoded inputs.
<a href="http://hunterheidenreich.com/blog/intro-to-word-embeddings/">word_embed blog</a>

<a href="https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa">word_embed blog II</a>


![W2V](./img/w2v.png)

##### Objective: maximize the sum of probabilities of each word given its observed window

This idea is very strong in comparison to other options: 

    Bag of words - just count occourences 
    TF-IDF - word is either informative or not but has no relation to other words
    one-hot encoder: for the computer paris-france is the same distance as paris-blabla

- Distance and direction are meaningful! King - man = Queen - woman
- Now the words massive and huge are similar!
- Extends to sentences, paragraphs and documents.

![](./img/see_attached_word_embed.png)

##### We have reduced the dimension of the vocublary by a big factor!
example: from 80,000 to 300

![](https://www.lemay.ai/demo/wordEmbedding/)

##### Insight I.I  The same thing can be applied to any categorical variable. 
##### With enough training data we can learn its continous position in space - state of the art

![categorical_embed](./img/categorical_embedding.png) # image I

![german_states](./img/german_states_mapped_2D.png) 

#### Insight II: 
        well, even if we can include many words (large n-gram), how can we capture context?
        If the text mentioned queen Mary and few pages later is talking about the queen, how will our network 
        know her name is Mary? 

### Idea I: Memory - your current choices are based on previous understanding

Add some cell in the network to keep previous memory and combine with current input to predict next word

![](./img/memory_rnn.png)

#### Problem: calculating the derivative (aka gradient) is problematic, either infinite or zero.

Imagine the memory at time t is the memory at time t-1 times a weight vector:
    $h_t = W*h_{t-1}$
Then:
    $h_t = W^t * h_0$ 
    
  $W > 1$ $h_t --> \infty$

### Solution: LSTM/GRU

<a href="https://colah.github.io/posts/2015-08-Understanding-LSTMs/">LSTM/GRU blog</a>


![](./img/RNNs.png)

![](./img/LSTM_colah.png)

#### Idea II: gates: don't multiply, use addition for memory!

##### Components

    - cell state
    - candidates  

##### Gates
- forget - information to throw (0 means throw all from the cell state)
- input - what values we are going to update
- output - filter which values of the cell we are going to output 

The current cell state is the sum of forgetting and updating with new candidates

### Extension: attention

<a href="https://www.youtube.com/watch?v=SysgYptB198">Intuition</a>

######  - Translate part by part
###### -  Use attention weights - how much attention should you give to each word in the input (update weights to each new word)

![](./img/attention.png)

### Takeaways:
    

##### Word embeddings
- Word/categorical embeddings gives meaning to words in relation to one another
- Word/categorical embeddings are computationally efficient
- Training is done through a classic NN with small window around words

##### RNN
- Old generation RNNs suffered from exploding/vanishing gradients
- New generation RNNs (commonly LSTM or GRU) are using memory gates to mitigate this problem
- RNNs are just multiple copies of a NN connected by the hidden layer
- Training is done again by backpropogation
- Weights are shared accros all network
- RNN's can be used for any sequence. Unlike time series models can include both time and features.
- Are flexible in input and output sizes
- Amazing results in NLP, recommendations and many more.
- Many flavours - BRNN, CRNN...

##### Attention
- Typicall for translations/images
- Weight all the words in one language to decide how much they should influence input to translated language
- components: word weights, BRNN, RNN, context vectors.

### Hands on Word2Vec/word embedding

In [26]:
!pip install gensim

In [27]:
import gensim
import numpy as np
import json
import string

##### Reading in the data

In [28]:
with open('/Users/OmersGuest/Documents/dc_ds_04_22_19/module_6/JEOPARDY_QUESTIONS1.json') as f:
    data = json.load(f)

In [29]:
len(data)

216930

In [30]:
# Let's look at the first element in our list
data[0]

{'category': 'HISTORY',
 'air_date': '2004-12-31',
 'question': "'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'",
 'value': '$200',
 'answer': 'Copernicus',
 'round': 'Jeopardy!',
 'show_number': '4680'}

In [31]:
# Word2Vec requires that our text have the form of a list
# of 'sentences', where each sentence is itself a list of
# words. How can we put our _Jeopardy!_ clues in that shape?

text = []

for clue in data:
    sentence = clue['question'].translate(str.maketrans('', '',
                                                        string.punctuation)).split(' ')
    
    new_sent = []
    for word in sentence:
        new_sent.append(word.lower())
    
    text.append(new_sent)

In [32]:
# Let's check the new structure of our first clue
text[0]

['for',
 'the',
 'last',
 '8',
 'years',
 'of',
 'his',
 'life',
 'galileo',
 'was',
 'under',
 'house',
 'arrest',
 'for',
 'espousing',
 'this',
 'mans',
 'theory']

#####  Constructing the model

In [33]:
# simply a matter of
# instantiating a Word2Vec object.
model = gensim.models.Word2Vec(text, sg=1)
## sg means skip-gram

##### training 

In [34]:
# To train, call 'train()'!
model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

(11335243, 15849970)

In [35]:
# The '.wv' attribute stores the word vectors
model.wv

In [36]:
model.wv['child']

array([-0.34274068,  0.5717052 ,  0.5054462 , -0.83763856,  0.11243444,
       -0.06243891, -0.22446074, -0.12599082,  0.4563118 ,  0.02078697,
       -0.33238268, -0.5581911 , -0.09881275,  0.43666995, -0.43092743,
        0.11493187,  0.01497902, -0.2810725 , -0.00601106, -0.56335247,
        0.04043306, -0.11607531, -0.05852357,  0.21723358,  0.05501913,
       -0.38965893,  0.16212374, -0.03791274,  0.13695273, -0.09983719,
       -0.03263056,  0.31049478,  0.07835225,  0.39296925, -0.06863066,
        0.07827546,  0.03597175,  0.39960814, -0.07369318,  0.17688808,
       -0.36443254,  0.3548273 ,  0.44067565,  0.20315555,  0.09774695,
       -0.4532549 , -0.20454371, -0.2576484 , -0.15994538,  0.19517948,
       -0.27781796,  0.16111617,  0.36601475,  0.25504816,  0.06936003,
        0.15185119, -0.70531076, -0.3037449 ,  0.14605772, -0.5506104 ,
       -0.285976  ,  0.33396873, -0.18368298,  0.01547174,  0.6061479 ,
        0.8693408 , -0.20910892, -0.19824041,  0.3992558 ,  0.00

In [37]:
### model.wv methods
#### 'most_similar()' and 'similarity()'

In [38]:
model.wv.most_similar('happiness')

[('wherefore', 0.7207859754562378),
 ('arise', 0.7125382423400879),
 ('virtue', 0.7093296051025391),
 ('despair', 0.7031866908073425),
 ('frailty', 0.703133225440979),
 ('kindness', 0.7010196447372437),
 ('autumns', 0.7005016207695007),
 ('loins', 0.6989527940750122),
 ('prosperity', 0.6966843008995056),
 ('shame', 0.6949119567871094)]

In [39]:
model.wv.most_similar('furniture')

[('chippendale', 0.7018810510635376),
 ('pottery', 0.7012165784835815),
 ('decorative', 0.6912299394607544),
 ('monogram', 0.6858183145523071),
 ('wearers', 0.6801064014434814),
 ('fastener', 0.672823965549469),
 ('ceramic', 0.6717427372932434),
 ('plaster', 0.6685993671417236),
 ('artwork', 0.6657806634902954),
 ('fasteners', 0.6614851355552673)]

In [40]:
model.wv.similarity('furniture', 'jewelry')

0.6603956

In [41]:
model.wv.most_similar(positive=['cat', 'animal', 'pet', 'mammal'])

[('reptile', 0.8154597878456116),
 ('carnivore', 0.8049635291099548),
 ('shorttailed', 0.7865055203437805),
 ('marsupial', 0.7851467728614807),
 ('rodent', 0.7808530330657959),
 ('feline', 0.7795917987823486),
 ('parrot', 0.7683272361755371),
 ('giraffe', 0.7652203440666199),
 ('venomous', 0.7638094425201416),
 ('burrowing', 0.7629961967468262)]

In [42]:
model.wv.most_similar(positive=['cat', 'animal'], negative='pet')

[('lizard', 0.3663201332092285),
 ('insect', 0.3660660982131958),
 ('sheep', 0.36604586243629456),
 ('domesticated', 0.35657936334609985),
 ('puppy', 0.3524196445941925),
 ('rodent', 0.3447427451610565),
 ('parrot', 0.344499796628952),
 ('breed', 0.3306526243686676),
 ('extinction', 0.3279828429222107),
 ('arachnid', 0.327955424785614)]

In [18]:
model.wv.most_similar(positive=['king', 'woman'], negative='man', topn=3)

[('throne', 0.297690749168396),
 ('monarch', 0.27509433031082153),
 ('queen', 0.2577328681945801)]

In [43]:
model.wv.most_similar(positive='usa')

[('fargo', 0.6092537641525269),
 ('purchasing', 0.6085797548294067),
 ('pageant', 0.6038713455200195),
 ('tyra', 0.6024805903434753),
 ('brisbane', 0.6017252802848816),
 ('310', 0.5972929000854492),
 ('retitled', 0.5966009497642517),
 ('coanchor', 0.5961899757385254),
 ('crossroads', 0.5891170501708984),
 ('dogpatch', 0.5890745520591736)]

In [20]:
model.wv.most_similar('canada')

[('zambia', 0.692581057548523),
 ('marianas', 0.6910848617553711),
 ('commonwealth', 0.6846738457679749),
 ('territories', 0.6586534976959229),
 ('uruguay', 0.6584171652793884),
 ('albania', 0.646291196346283),
 ('manitoba', 0.6458621621131897),
 ('yalta', 0.6444039940834045),
 ('chile', 0.6434510946273804),
 ('liechtenstein', 0.6419185996055603)]

In [44]:
model.wv.most_similar('shakespeare')

[('shakespeares', 0.7306967973709106),
 ('sophocles', 0.7042598724365234),
 ('romeo', 0.7032291889190674),
 ('euripides', 0.7012650966644287),
 ('falstaff', 0.6938175559043884),
 ('shakespearean', 0.6827644109725952),
 ('ibsen', 0.6718511581420898),
 ('hamlet', 0.6689773201942444),
 ('shaws', 0.659666121006012),
 ('bards', 0.6557680368423462)]

In [46]:
model.wv.most_similar(positive=['president', 'germany'], negative='usa')

[('emperors', 0.24682524800300598),
 ('dictator', 0.24275463819503784),
 ('ussr', 0.21163707971572876),
 ('nazi', 0.2034521847963333),
 ('emperor', 0.20158304274082184),
 ('versailles', 0.19930800795555115),
 ('invaded', 0.19467172026634216),
 ('russia', 0.19159460067749023),
 ('shah', 0.18833297491073608),
 ('officially', 0.18761301040649414)]

In [23]:
#### 'doesnt_match()'

In [45]:
model.wv.doesnt_match(['breakfast', 'lunch', 'frog', 'food'])

/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'frog'

More RNN: 